# Vigilancia epidemiológica ANUAL

Adaptado de https://github.com/mauforonda/vigilancia-epidemiologica/blob/master/vigilancia-epidemiologica.ipynb 
Informacion obtenida de la base de datos online del Sistema Nacional de Informacion de Salud del Ministerio de Salud y Deportes- Bolivia (https://estadisticas.minsalud.gob.bo/Default_Vigilancia.aspx) registrada y año entre 2001 y 2021.

In [ ]:
# dependencias

import pandas as pd
import requests
import datetime as dt
from bs4 import BeautifulSoup
import unicodedata
from slugify import slugify
from IPython.display import clear_output
from itables import show, init_notebook_mode
init_notebook_mode()
from IPython.display import display, IFrame
import itertools
import re
import json

__Variables guardadas__: usando comando magico `%store`
* Guardar múltiples variables en la base de datos de IPython: %store var1 var2
* Actualización de los valores de una o varias variables: %store -r var1
* Listado de todas las variables almacenadas en la base de datos junto a sus valres: %store
* Borrado de una o varias variables de la base de datos: %store -d var1
* Borrado de todas las variables de la base de datos: %store -z
* Guardado de las variables en un archivo %store var1 > var1.txt

In [ ]:
%store

A continuación detallo el proceso de construcción de estos datos, que consiste en 3 pasos:

1. Construir un inventario de variables
2. Descargar cada datos disponible
3. Preparar los datos para que sean utilizados fácilmente

## Un inventario de variables

*Qué información está disponible?*

El ministerio construye una página diferente para mostrar los datos de cada año. Es necesario consultar cada página y navegar los menús que ofrece para listar las variables y grupos de variables que exhibe. Algo interesante de estos tableros es que para entregar información, además de los parámetros que describen la consulta e identidad del usuario, requieren valores que describen el estado de navegación del usuario en el tablero durante la consulta anterior. Es decir que para mostrar una tabla en particular es necesario no sólo ejecutar la consulta correcta, sino la secuencia previa de consultas correctas que el servicio espera. Por suerte, estas limitaciones son fáciles de resolver.

In [2]:
# Información que el servidor espera de un usuario normal

cookies = {
    'ASP.NET_SessionId': 'es15y505g3h0e14ooob5ebns', # Place your own 🍪 🍪 🍪
}

headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:98.0) Gecko/20100101 Firefox/98.0',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.5',
    'Origin': 'https://estadisticas.minsalud.gob.bo',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-User': '?1',
    'Pragma': 'no-cache',
    'Cache-Control': 'no-cache',
}

In [3]:
# Some helper dicts
#En la fec/a de consulta los año 12/08/2023 los años2001-2004 no estan disponibles
#ya llamé y escribi una carta
# me contestaroon que cuando lo "aprueben"
years = [
#    {'year': 2001, 'pagina': 'Form_Vigi_2001.aspx'},
 #   {'year': 2002, 'pagina': 'Form_Vigi_2001.aspx'},
  #  {'year': 2003, 'pagina': 'Form_Vigi_2001.aspx'},
   # {'year': 2004, 'pagina': 'Form_Vigi_2001.aspx'},
    {'year': 2005, 'pagina': 'Form_Vigi_2007.aspx'},
    {'year': 2006, 'pagina': 'Form_Vigi_2007.aspx'},
    {'year': 2007, 'pagina': 'Form_Vigi_2007.aspx'},
    {'year': 2008, 'pagina': 'Form_Vigi_2008.aspx'},
    {'year': 2009, 'pagina': 'Form_Vigi_2009.aspx'},
    {'year': 2010, 'pagina': 'Form_Vigi_2010.aspx'},
    {'year': 2011, 'pagina': 'Form_Vigi_2011.aspx'},
    {'year': 2012, 'pagina': 'Form_Vigi_2012.aspx'},
    {'year': 2013, 'pagina': 'Form_Vigi_2013.aspx'},
    {'year': 2014, 'pagina': 'Form_Vigi_2014_2.aspx'},
    {'year': 2015, 'pagina': 'Form_Vigi_2015_302a.aspx'},
    {'year': 2016, 'pagina': 'Form_Vigi_2016_302a.aspx'},
    {'year': 2017, 'pagina': 'Form_Vigi_2017_302a.aspx'},
    {'year': 2018, 'pagina': 'Form_Vigi_2018_302a.aspx'},
    {'year': 2019, 'pagina': 'Form_Vigi_2019_302a.aspx'},
    {'year': 2020, 'pagina': 'Form_Vigi_2020_302a.aspx'},
    {'year': 2021, 'pagina': 'Form_Vigi_2021_302a.aspx'}
]

state = {}

paginas = {y['year']:y['pagina'] for y in years}

In [4]:
# Functions to build a list of available datasets
#CAMBIANDO EL VALOR DE WebPanel2$seleccion a '2' para seleccionar todos los municipios incluyendo los sin casos

def get_subvars(url, year, grvar):
    """
    Makes a list of every variable under a variable group and year.
    """
    
    data = {
        'ctl00$MainContent$WebPanel2_hidden': '',
        '__EVENTTARGET': 'ctl00$MainContent$WebPanel2$List_grvar',
        '__EVENTARGUMENT': '',
        '__LASTFOCUS': '',
        'ctl00$MainContent$WebPanel3_hidden': '%3CWebPanel%20Expanded%3D%22false%22%3E%3C/WebPanel%3E',
        'ctl00$MainContent$WebPanel2$List_gestion': str(year),
        'ctl00$MainContent$WebPanel2$List_fomulario': '302',
        'ctl00$MainContent$WebPanel2$Grupo': 'nomDepto',
        'ctl00$MainContent$WebPanel2$seleccion': '2',#esto tiene que cambiar a 2
        'ctl00$MainContent$WebPanel2$List_grvar': grvar
    }

    data = {**data, **state}
    
    response = requests.post(url, cookies=cookies, headers=headers, data=data)
    html = BeautifulSoup(response.text, 'html.parser')
    for node in ['__VIEWSTATE', '__VIEWSTATEGENERATOR', '__EVENTVALIDATION']:
        state[node] = html.select('#{}'.format(node))[0]['value']
    subvars = [{'subvar': option['value'], 'variable': option.get_text().strip()} for option in html.select('#MainContent_WebPanel2_Lista_subvar option')]
    return subvars


In [5]:
def get_variables(year, pagina):
    """
    Makes a list of all variables and variable groups for a year. 
    """
    
    data = {
        'ctl00$MainContent$WebPanel2_hidden': '',
        '__EVENTTARGET': 'ctl00$MainContent$WebPanel2$List_gestion',
        '__EVENTARGUMENT': '',
        '__LASTFOCUS': '',
        'ctl00$MainContent$WebPanel3_hidden': '%3CWebPanel%20Expanded%3D%22false%22%3E%3C/WebPanel%3E',
        'ctl00$MainContent$WebPanel2$List_gestion': str(year),
        'ctl00$MainContent$WebPanel2$List_fomulario': '302',
        'ctl00$MainContent$WebPanel2$List_grvar': '01',
        'ctl00$MainContent$WebPanel2$Grupo': 'nomDepto',
        'ctl00$MainContent$WebPanel2$seleccion': '2', #cambiando a valor '2' = ambos
    }
    
    url = 'https://estadisticas.minsalud.gob.bo/Reportes_Vigilancia/{}'.format(pagina)
    variables[year] = []
    response = requests.get(url, cookies=cookies, headers=headers)
    html = BeautifulSoup(response.text, 'html.parser')
    for node in ['__VIEWSTATE', '__VIEWSTATEGENERATOR', '__EVENTVALIDATION']:
        state[node] = html.select('#{}'.format(node))[0]['value']
    response = requests.post(url, cookies=cookies, headers=headers, data={**data, **state})   
    html = BeautifulSoup(response.text, 'html.parser')
    for node in ['__VIEWSTATE', '__VIEWSTATEGENERATOR', '__EVENTVALIDATION']:
        state[node] = html.select('#{}'.format(node))[0]['value']
    for option in html.select('#MainContent_WebPanel2_List_grvar option'):
        grupo = option.get_text().strip()
        grvar = option['value']
        subvars = get_subvars(url, year, grvar)
        for v in subvars:
            variables[year].append({'grvar': grvar, 
                                    'grupo': grupo,
                                    'subvar': v['subvar'],
                                    'variable': v['variable']})


In [6]:
def format_variables(variables):
    """
    Format variables and variable groups so that they're meaningful and easier to harmonize in the future.
    """

    dfv = []
    for y in variables.keys():
        dfi = pd.DataFrame(variables[y])
        dfi.insert(0, 'year', y)
        dfv.append(dfi)
    dfv = pd.concat(dfv)

    dfv.variable = dfv.variable.str.replace('^[0-9\.\-]+ ', '', regex=True)
    dfv.variable = dfv.variable.str.lower()
    dfv.variable = dfv.variable.apply(lambda x: unicodedata.normalize('NFKD', x).encode('ascii', 'ignore').decode('ascii'))

    dfv.grupo = dfv.grupo.str.lower()
    dfv.grupo = dfv.grupo.apply(lambda x: unicodedata.normalize('NFKD', x).encode('ascii', 'ignore').decode('ascii'))
    
    return dfv.reset_index(drop=True)

In [48]:
# Make a list of all variables and variable groups for every year and format them right

variables = {}
for year in years:
    print(year['year'])
    get_variables(year['year'], year['pagina'])

dfv = format_variables(variables)

2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021


In [8]:
# save
dfv.to_csv('resources/variables.csv', index=False)


Encuentro más de 1500 variables que podría consultar entre 2001 y 2021. El inventario se ve así:
#nota personal los datos del 2001 -2004 estan caidos actualmente hay 1232 en total

In [9]:
dfv

,year,grvar,grupo,subvar,variable
0,2005,01,sospecha diagnostica,01,sarampion / rubeola
1,2005,01,sospecha diagnostica,03,tos ferina
2,2005,01,sospecha diagnostica,04,difteria
3,2005,01,sospecha diagnostica,05,paralisis flacida aguda
4,2005,01,sospecha diagnostica,06,fiebre amarilla
...,...,...,...,...,...
1227,2021,19,"mortalidad perinatal, neonatal e infantil",02,muerte neonatal temprana (0-6 dias) confirmada
1228,2021,19,"mortalidad perinatal, neonatal e infantil",03,muerte neonatal tardia (7-27 dias) confirmada
1229,2021,19,"mortalidad perinatal, neonatal e infantil",04,muerte menor de 28 dias por sepsis
1230,2021,19,"mortalidad perinatal, neonatal e infantil",05,muerte menor de 28 dias por asfixia perinatal


## VARIABLES Seleccionadas
usando el dataframe que es guardado como variables.csv elijo manualmente (no hay de otra) las variables cuyos datos deseo bajar y las guardo en un nuevo csv llamado variables_seleccionades. Cabe decir que las columnas correspondientes a _`grvar`_ y _`subvar`_ al abrirse en Excel son leidas como numero y no como texto por lo cual pierden el 0 que le antecede. Es necesario trabajar con un editor que evite esta pérdida.

In [49]:
#hay que modificar ciertos caracteres: "", / + - () de los nombres que provocan error al descargar los archivos 
# Ruta al archivo CSV
ruta_csv = 'resources/variables_filtradas.csv'

# Lee el archivo CSV y especifica el tipo de dato de las columnas grvar y subvar como texto
dfv2 = pd.read_csv(ruta_csv, dtype={'grvar': str, 'subvar': str})

# Ahora puedes trabajar con 'data_frame', que es un objeto DataFrame de Pandas
dfv2


,year,grvar,grupo,subvar,variable
0,2005,01,sospecha diagnostica,03,tos ferina
1,2005,01,sospecha diagnostica,04,difteria
2,2005,02,sospecha diagnostica,09,fiebre hemorrag. bol
3,2005,02,sospecha diagnostica,14,enfer. por hanta virus
4,2005,06,otras infecciones,02,ira sin neumonia
...,...,...,...,...,...
193,2021,06,otras infecciones,12,neumonia
194,2021,09,mortalidad,10,muerte menor de 5 anos por neumonia
195,2021,17,enfermedades transmitidas por vectores (etv),01,leishmaniasis
196,2021,17,enfermedades transmitidas por vectores (etv),02,chagas agudo


En esta tabla, los valores `grvar` y `subvar` son códigos que describen un grupo de variables y variables en el tablero de un año `year`. Estos códigos serán útiles para solicitar datos al servidor en la próxima sección. Las condiciones, códigos y forma de los datos varían demasiado entre distintos años. Por eso, para tener un proceso que funcione a través de todas estas condiciones y años es importante sostener la menor cantidad de supuestos sobre cómo realizar consultas correctas o cómo se ve una tabla específica.

## Datos

*Descargar todos los datos disponibles a nivel ANUAL y municipal.*

Con el inventario en mano, el próximo paso es la descarga. Realizo consultas para cada variable, año y mes, que almaceno en documentos csv, uno para cada variable y año. Las variables y años están bien armonizadas gracias al índice. Y respecto al municipio, para tener datos que puedo cruzar fácilmente con otros sets de datos en la próxima sección construyo un diccionario que me permite armonizar códigos para municipios a través de todos los años.

In [50]:
#CAMBIANDO EL VALOR DE WebPanel2$seleccion a '2' para seleccionar todos los municipios incluyendo los sin casos

def get_state(url, year):
    """
    Inicializa parámetros de estado para persuadir al sistema a que nos entregue datos
    """
    
    data = {
        'ctl00$MainContent$WebPanel2_hidden': '',
        '__EVENTTARGET': 'ctl00$MainContent$WebPanel2$List_gestion',
        '__EVENTARGUMENT': '',
        '__LASTFOCUS': '',
        'ctl00$MainContent$WebPanel3_hidden': '%3CWebPanel%20Expanded%3D%22false%22%3E%3C/WebPanel%3E',
        'ctl00$MainContent$WebPanel2$List_gestion': str(year),
        'ctl00$MainContent$WebPanel2$List_fomulario': '302',
        'ctl00$MainContent$WebPanel2$List_grvar': '01',
        'ctl00$MainContent$WebPanel2$Grupo': 'nomDepto',
        'ctl00$MainContent$WebPanel2$seleccion': '2', #cambiando a valor 2 = ambos
    }
    
    response = requests.get(url, cookies=cookies, headers=headers)
    html = BeautifulSoup(response.text, 'html.parser')
    for node in ['__VIEWSTATE', '__VIEWSTATEGENERATOR', '__EVENTVALIDATION']:
        state[node] = html.select('#{}'.format(node))[0]['value']
        
    response = requests.post(url, cookies=cookies, headers=headers, data={**data, **state})   
    html = BeautifulSoup(response.text, 'html.parser')
    for node in ['__VIEWSTATE', '__VIEWSTATEGENERATOR', '__EVENTVALIDATION']:
        state[node] = html.select('#{}'.format(node))[0]['value']
        

##ESTA FUNCION CAMBIA PARA OBTENER SOLO VALORES ANUALES
def update_state(url, year, grvar):
    """
    Actualiza parámetros de estado para persuadir al sistema a que nos entregue datos
    """

    data = {

        'ctl00$MainContent$WebPanel2_hidden': '',
        '__EVENTTARGET': 'ctl00$MainContent$WebPanel2$List_grvar',
        '__EVENTARGUMENT': '',
        '__LASTFOCUS': '',
        'ctl00$MainContent$WebPanel3_hidden': '%3CWebPanel%20Expanded%3D%22false%22%3E%3C/WebPanel%3E',
        'ctl00$MainContent$WebPanel2$List_gestion': str(year),
        'ctl00$MainContent$WebPanel2$List_fomulario': '302',
        'ctl00$MainContent$WebPanel2$Grupo': 'nomMunicip',
        'ctl00$MainContent$WebPanel2$seleccion': '2', #cambiando valor a 2 para todos los municipios
       #'ctl00$MainContent$WebPanel2$List_mes': '1',
        'ctl00$MainContent$WebPanel2$List_grvar': grvar
    }
    data = {**data, **state}
    
    response = requests.post(url, cookies=cookies, headers={**headers, **{'Referer': url}}, data=data)
    html = BeautifulSoup(response.text, 'html.parser')
    for node in ['__VIEWSTATE', '__VIEWSTATEGENERATOR', '__EVENTVALIDATION']:
        state[node] = html.select('#{}'.format(node))[0]['value']

#QUITAMOS EL MES Y DEJAMOS EL RESTO --> def get_month(url, year, mes, grvar, subvar):
def get_month(url, year, grvar, subvar):

    """
    Descarga datos para un mes en una variable y año.
    """

    data = {
        'ctl00$MainContent$WebPanel2_hidden': '',
        '__EVENTTARGET': '',
        '__EVENTARGUMENT': '',
        '__LASTFOCUS': '',
        'ctl00$MainContent$WebPanel3_hidden': '%3CWebPanel%20Expanded%3D%22false%22%3E%3C/WebPanel%3E',
        'ctl00$MainContent$WebPanel2$List_gestion': str(year),
        'ctl00$MainContent$WebPanel2$List_fomulario': '302',
        'ctl00$MainContent$WebPanel2$Grupo': 'nomMunicip',
        'ctl00$MainContent$WebPanel2$seleccion': '2', #cambiando el valor a 2 para todos los municipios
        'ctl00$MainContent$WebPanel2$Button1': ' Procesar',
        #'ctl00$MainContent$WebPanel2$List_mes': str(mes),
        'MainContentxWebPanel3xmydatagrid': '',
        'MainContentxWebPanel3xmydatagrid2': '',
        'ctl00$MainContent$WebPanel2$List_grvar': grvar,
        'ctl00$MainContent$WebPanel2$Lista_subvar': subvar
    }
    data = {**data, **state}
    
    response = requests.post(url, cookies=cookies, headers={**headers, **{'Referer': url}}, data=data)
    html = BeautifulSoup(response.text, 'html.parser')
    table = html.select('#G_MainContentxWebPanel3xmydatagrid')[0]
    
    df = pd.read_html(str(table))[0]
    df = df.set_index(df.columns[0])
    df = df[~df.index.str.lower().str.contains('total')]
    
    return df
    

def get_dataset(year, grvar, subvar, grupo, variable):
    """
    Descarga datos para cada mes en una variable y año.
    """
    
    year_data = []
    url = 'https://estadisticas.minsalud.gob.bo/Reportes_Vigilancia/{}'.format(paginas[year])
    get_state(url, year)
    update_state(url, year, grvar)
    #El for no es mas necesario para el año
    #for mes in range(1,13):
    #    mes_data = get_month(url, year, mes, grvar, subvar)
    #    mes_data.insert(0, 'mes', dt.date(year, mes, 1))
    #    year_data.append(mes_data)
    mes_data = get_month(url, year, grvar, subvar)
    mes_data.insert(0, 'year', dt.date(year, 1, 1).year)##ESO ME PERMITE GUARDAR SOLO EL ANHO
    year_data.append(mes_data)
    year_data = pd.concat(year_data)
    year_data.insert(0, 'variable', variable)
    year_data.insert(0, 'grupo', grupo)
    filename = 'datos/{}_{}_{}.csv'.format(year, slugify(grupo), slugify(variable))
    indice.append(dict(year=year, filename=filename, grvar=grvar, grupo=grupo, subvar=subvar, variable=variable))
    year_data.to_csv(filename)

In [39]:
##USANDO DATA FRAME dfv2 CON LAS VARIABLES SELECCIONADAS
indice = []
# Define el número máximo de iteraciones para probar
max_iterations = 2  # Cambia esto al número que desees

# Descargar datos para cada fila en el inventario y simultáneamente construir un índice
for i, row in dfv2.iterrows():
    if i >= max_iterations:
     break  # Detener la ejecución después de alcanzar el número máximo de iteraciones

    clear_output(wait=True)
    print('{}/{} : {} en {}'.format(i+1, len(dfv2), row['variable'], row['year']))
    get_dataset(row['year'], row['grvar'], row['subvar'], row['grupo'], row['variable'])

indice = pd.DataFrame(indice)
indice.to_csv('resources/indice.csv', index=False)

2/198 : difteria en 2005


In [51]:
# Descargar datos para cada fila en el inventario y simultáneamente construir un índice
indice =[]
for i, row in dfv2.iterrows():
    
    clear_output(wait=True)
    print('{}/{} : {} en {}'.format(i+1, len(dfv2), row['variable'], row['year']))
    get_dataset(row['year'], row['grvar'], row['subvar'], row['grupo'], row['variable'])

indice = pd.DataFrame(indice)
indice.to_csv('resources/indice.csv', index=False)

198/198 : malaria en 2021


In [52]:
indice
#recien sale bien los nombres asi: datos/2005_sospecha-etc...

,year,filename,grvar,grupo,subvar,variable
0,2005,datos/2005_sospecha-diagnostica_tos-ferina.csv,01,sospecha diagnostica,03,tos ferina
1,2005,datos/2005_sospecha-diagnostica_difteria.csv,01,sospecha diagnostica,04,difteria
2,2005,datos/2005_sospecha-diagnostica_fiebre-hemorra...,02,sospecha diagnostica,09,fiebre hemorrag. bol
3,2005,datos/2005_sospecha-diagnostica_enfer-por-hant...,02,sospecha diagnostica,14,enfer. por hanta virus
4,2005,datos/2005_otras-infecciones_ira-sin-neumonia.csv,06,otras infecciones,02,ira sin neumonia
...,...,...,...,...,...,...
193,2021,datos/2021_otras-infecciones_neumonia.csv,06,otras infecciones,12,neumonia
194,2021,datos/2021_mortalidad_muerte-menor-de-5-anos-p...,09,mortalidad,10,muerte menor de 5 anos por neumonia
195,2021,datos/2021_enfermedades-transmitidas-por-vecto...,17,enfermedades transmitidas por vectores (etv),01,leishmaniasis
196,2021,datos/2021_enfermedades-transmitidas-por-vecto...,17,enfermedades transmitidas por vectores (etv),02,chagas agudo


Son consultas secuenciales, en parte porque no quiero arriesgar degradar la disponibilidad del sitio y también porque es el proceso más sencillo de programar y evaluar. En total debe haber tomado alrededor de 9 horas de consultas y, si bien el proceso es reproducible, espero que no sea algo que deba hacerse de manera rutinaria.

## Limpieza

*Que cada tabla tenga una forma predecible y municipios armonizados.*

Finalmente, pongo los datos en una estructura simple y extensible. Las primeras columnas de todos los sets de datos describen el código INE del municipio, el nombre del municipio que ofrece el tablero, el grupo de variable, el nombre de la variable y el mes de los valores. Para tener los códigos INE, construyo un diccionario que mapea cada nombre de municipio con su código, utilizando las tablas que publica el ministerio de salud sobre la estructura de establecimientos entre 2005 y 2021. Por suerte, los nombres en estas tablas y la base de datos de vigilancia epidemiológica son las mismas, debido a que el reporte se suele realizar mediante un sistema que consume listas similares publicadas periódicamente. El resto de las columnas consisten en todos los datos que ofrece el sistema, a veces desagregados por rangos de edad, sexo, si la atención fue prestada en el establecimiento o fuera, etc. No hago más que la limpieza más simple sobre estas columnas, suficiente como para que sean utilizadas rápidamente en el futuro y sin muchos juicios sobre cómo deberían verse. Con esta información es fácil agregar valores de condiciones a través de varios años y cruzarlos con información a nivel municipal, como por ejemplo indicadores de los Objetivos de Desarrollo Sostenible.

In [53]:
##HEMOS MODIFICADO PARA QUE LOS DATOS SEAN ANUALES
#esta funcion tiene ahora para colocar la columna departamento
def format_vigilancia(indice_seleccion, municipio_dict, depto_dict):
    
    errores = []
    len_seleccion = len(indice_seleccion)

    for i, f in enumerate(indice_seleccion.filename):
        clear_output(wait=True)
        print('{} / {}'.format(i+1, len_seleccion))
        try:

            dfi = pd.read_csv('{}'.format(f))
            
            #if dfi.mes.isna().sum() > 0:
            if dfi.year.isna().sum() > 0:   
            #   header = dfi[dfi.mes.isna()][dfi.columns[4:]].to_dict()
                header = dfi[dfi.year.isna()][dfi.columns[4:]].to_dict()
                header = ['_'.join([slugify(k)] + [slugify(header[k][kk]) for kk in header[k].keys() if type(header[k][kk]) == str ]) for k in header.keys()]
                dfi = dfi[dfi.year.notna()]
            #   dfi = dfi[dfi.mes.notna()]
            #   dfi.columns = ['municipio', 'grupo', 'variable', 'mes'] + header
                dfi.columns = ['municipio', 'grupo', 'variable', 'year'] + header


            else:
                dfi = dfi.rename(columns={'Municipio':'municipio'})
            
            dfi.insert(0, 'codigo_municipio', dfi.municipio.map(municipio_dict))
                       
           #AQUI INSERTAMOS EL DEPTO
            dfi.insert(0, 'DEPARTAMENTO', dfi.municipio.map(depto_dict))#tambien lo ingresamos en la funcion arriba
            dfi.to_csv('datos/{}'.format(f.split('/')[1]), index=False)

        except Exception as e:
            errores.append({'filename': f, 'error': e})

    return errores

#AQUI NO VARIA NADA PARA ANUAL
##aqui hacemos modificaciones PARA INGRESAR EL DEPARTAMENTO
def listar_municipios1(f):
    
    municipios_snis = []

    establecimientos = pd.ExcelFile(f)

    for s in establecimientos.sheet_names:
        e = pd.read_excel(establecimientos, sheet_name=s, header=2)
        e = e[['COD_MUNICIPIO', 'MUNICIPIO','DEPARTAMENTO']]#modifcado 
        e = e.drop_duplicates()
        municipios_snis.append(e)

    municipios_snis = pd.concat(municipios_snis)
    municipios_snis = municipios_snis[municipios_snis.COD_MUNICIPIO.notna()]
    municipios_snis['COD_MUNICIPIO'] = municipios_snis.COD_MUNICIPIO.astype(int)
    municipios_snis['MUNICIPIO'] = municipios_snis.MUNICIPIO.str.strip()
    municipios_snis['DEPARTAMENTO'] = municipios_snis.DEPARTAMENTO.str.strip()##MODIFICADO
    municipios_snis = municipios_snis.drop_duplicates()
    
    return municipios_snis


def listar_municipios2(f):
    e = pd.read_excel(f, sheet_name='BASE DE DATOS', header=3)
    e = e[['COD_MUN', 'MUN','DPTO']]##MODIFIED
    e.columns = ['COD_MUNICIPIO', 'MUNICIPIO','DEPARTAMENTO']
    e = e[e.MUNICIPIO.notna()]
    e['COD_MUNICIPIO'] = e.COD_MUNICIPIO.astype(int)
    e['MUNICIPIO'] = e.MUNICIPIO.str.strip()
    e['DEPARTAMENTO'] = e.DEPARTAMENTO.str.strip()#modified
    e = e.drop_duplicates()
    return e

In [15]:
# Construir un diccionario para armonizar municipios

municipios_snis = pd.concat([
    listar_municipios1('resources/Establecimientos 2005_2017.xlsx'), 
    listar_municipios2('resources/ESTRUCTURA DE EE.SS. GESTION 2021_DASHBOARDcerrado oficial.xlsx')
]).drop_duplicates()

municipios_snis['MUNICIPIO'] = municipios_snis.MUNICIPIO.apply(lambda x: ' '.join(x.split()))
municipios_dict = municipios_snis.set_index('MUNICIPIO').COD_MUNICIPIO.to_dict()


In [16]:
#INGRESO UN NUEVO DICCIONARIO CON LOS DEPARTAMENTOS
departamentos_dict = municipios_snis.set_index('MUNICIPIO').DEPARTAMENTO.to_dict()

In [17]:
municipios_snis

,COD_MUNICIPIO,MUNICIPIO,DEPARTAMENTO
0,10201,AZURDUY,CHUQUISACA
7,10202,TARVITA,CHUQUISACA
16,10403,SOPACHUY,CHUQUISACA
19,10701,CAMARGO,CHUQUISACA
26,10702,SAN LUCAS,CHUQUISACA
...,...,...,...
760,20205,SANTIAGO DE HUATA,LA PAZ
763,20206,HUATAJATA,LA PAZ
1257,31304,RAQAYPAMPA(ETA),COCHABAMBA
1574,20803,TIAHUANACU,LA PAZ


In [43]:
municipios_dict

{'AZURDUY': 10201,
 'TARVITA': 10202,
 'SOPACHUY': 10403,
 'CAMARGO': 10701,
 'SAN LUCAS': 10702,
 'INCAHUASI': 10703,
 'CAMATAQUI (C. VILLA ABECIA)': 10901,
 'CULPINA': 10902,
 'LAS CARRERAS': 10903,
 'MONTEAGUDO': 10501,
 'HUACARETA': 10502,
 'VILLA VACA GUZMAN (MUYUPAMPA)': 11001,
 'HUACAYA (VILLA DE HUACAYA)': 11002,
 'MACHARETI': 11003,
 'PADILLA': 10401,
 'TOMINA': 10402,
 'ALCALA': 10404,
 'EL VILLAR': 10405,
 'VILLA SERRANO': 10801,
 'SUCRE': 10101,
 'POROMA': 10103,
 'YOTALA': 10102,
 'ZUDAÑEZ': 10301,
 'PRESTO': 10302,
 'MOJOCOYA': 10303,
 'ICLA': 10304,
 'TARABUCO': 10601,
 'YAMPARAEZ': 10602,
 'EL ALTO': 20105,
 'LA PAZ': 20101,
 'IXIAMAS': 21501,
 'SAN BUENA VENTURA': 21502,
 'CHUMA': 20501,
 'AYATA': 20502,
 'AUCAPATA': 20503,
 'APOLO': 20701,
 'PELECHUCO': 20702,
 'GRAL. J.J. PEREZ (CHARAZANI)': 21601,
 'CURVA': 21602,
 'PUERTO ACOSTA': 20401,
 'MOCOMOCO': 20402,
 'PUERTO CARABUCO': 20403,
 'ACHACACHI': 20201,
 'ANCORAIMES': 20202,
 'SORATA': 20601,
 'TACACOMA': 20603,
 

In [19]:
#funciono!
departamentos_dict

{'AZURDUY': 'CHUQUISACA',
 'TARVITA': 'CHUQUISACA',
 'SOPACHUY': 'CHUQUISACA',
 'CAMARGO': 'CHUQUISACA',
 'SAN LUCAS': 'CHUQUISACA',
 'INCAHUASI': 'CHUQUISACA',
 'CAMATAQUI (C. VILLA ABECIA)': 'CHUQUISACA',
 'CULPINA': 'CHUQUISACA',
 'LAS CARRERAS': 'CHUQUISACA',
 'MONTEAGUDO': 'CHUQUISACA',
 'HUACARETA': 'CHUQUISACA',
 'VILLA VACA GUZMAN (MUYUPAMPA)': 'CHUQUISACA',
 'HUACAYA (VILLA DE HUACAYA)': 'CHUQUISACA',
 'MACHARETI': 'CHUQUISACA',
 'PADILLA': 'CHUQUISACA',
 'TOMINA': 'CHUQUISACA',
 'ALCALA': 'CHUQUISACA',
 'EL VILLAR': 'CHUQUISACA',
 'VILLA SERRANO': 'CHUQUISACA',
 'SUCRE': 'CHUQUISACA',
 'POROMA': 'CHUQUISACA',
 'YOTALA': 'CHUQUISACA',
 'ZUDAÑEZ': 'CHUQUISACA',
 'PRESTO': 'CHUQUISACA',
 'MOJOCOYA': 'CHUQUISACA',
 'ICLA': 'CHUQUISACA',
 'TARABUCO': 'CHUQUISACA',
 'YAMPARAEZ': 'CHUQUISACA',
 'EL ALTO': 'LA PAZ',
 'LA PAZ': 'LA PAZ',
 'IXIAMAS': 'LA PAZ',
 'SAN BUENA VENTURA': 'LA PAZ',
 'CHUMA': 'LA PAZ',
 'AYATA': 'LA PAZ',
 'AUCAPATA': 'LA PAZ',
 'APOLO': 'LA PAZ',
 'PELECHUCO'

In [54]:
#nuevamente una mirada al indice
indice

,year,filename,grvar,grupo,subvar,variable
0,2005,datos/2005_sospecha-diagnostica_tos-ferina.csv,01,sospecha diagnostica,03,tos ferina
1,2005,datos/2005_sospecha-diagnostica_difteria.csv,01,sospecha diagnostica,04,difteria
2,2005,datos/2005_sospecha-diagnostica_fiebre-hemorra...,02,sospecha diagnostica,09,fiebre hemorrag. bol
3,2005,datos/2005_sospecha-diagnostica_enfer-por-hant...,02,sospecha diagnostica,14,enfer. por hanta virus
4,2005,datos/2005_otras-infecciones_ira-sin-neumonia.csv,06,otras infecciones,02,ira sin neumonia
...,...,...,...,...,...,...
193,2021,datos/2021_otras-infecciones_neumonia.csv,06,otras infecciones,12,neumonia
194,2021,datos/2021_mortalidad_muerte-menor-de-5-anos-p...,09,mortalidad,10,muerte menor de 5 anos por neumonia
195,2021,datos/2021_enfermedades-transmitidas-por-vecto...,17,enfermedades transmitidas por vectores (etv),01,leishmaniasis
196,2021,datos/2021_enfermedades-transmitidas-por-vecto...,17,enfermedades transmitidas por vectores (etv),02,chagas agudo


In [55]:
errores = format_vigilancia(indice, municipios_dict, departamentos_dict)

198 / 198


In [56]:
errores

[]

## Para terminar, así es como se ve uno de los 198 sets de datos escogido aleatoriamente:

In [57]:
#si se pudo anual!!
#aqui la tabla deberia salir con todos municipios con casos y sin casos
pd.read_csv('{}'.format(indice.sample()['filename'].iloc[0]))


,DEPARTAMENTO,codigo_municipio,municipio,grupo,variable,year,a-menor-de-6-meses_masculino,a-menor-de-6-meses-1_femenino,b-6-meses-a-menor-de-1-ano_masculino,b-6-meses-a-menor-de-1-ano-1_femenino,...,g-20-39-anos-1_femenino,h-40-49-anos_masculino,h-40-49-anos-1_femenino,i-50-59-anos_masculino,i-50-59-anos-1_femenino,j-60-anos-y-mas_masculino,j-60-anos-y-mas-1_femenino,totales_dentro,totales-1_fuera,totales-2_total
0,POTOSI,51302,ACACIO,sospecha diagnostica,difteria,2021.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,LA PAZ,20201,ACHACACHI,sospecha diagnostica,difteria,2021.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,LA PAZ,20104,ACHOCALLA,sospecha diagnostica,difteria,2021.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,COCHABAMBA,30201,AIQUILE,sospecha diagnostica,difteria,2021.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,COCHABAMBA,31303,ALALAY,sospecha diagnostica,difteria,2021.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336,POTOSI,50103,YOCALLA,sospecha diagnostica,difteria,2021.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
337,CHUQUISACA,10102,YOTALA,sospecha diagnostica,difteria,2021.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
338,TARIJA,60402,YUNCHARA,sospecha diagnostica,difteria,2021.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
339,ORURO,40504,YUNGUYO DE LITORAL,sospecha diagnostica,difteria,2021.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Reformateo de nombres

Cada set de datos representa el número de casos clasificados en una variable para un año, desagregado por meses y municipios. Si bien una misma variable es monitoreada en múltiples periodos, sus nombres y las formas de la tabla que produce el sistema suelen cambiar.

In [59]:
indice = pd.read_csv('resources/indice.csv',dtype={'grvar': str, 'subvar': str})#no olvidarse espeificar el type de datos que son grvar y subvar
unicos = indice[['grvar', 'subvar', 'grupo', 'variable']].drop_duplicates(subset=['grvar', 'subvar', 'variable']).reset_index(drop=True)
unicos = unicos[unicos.variable.notna()]

In [60]:
unicos

,grvar,subvar,grupo,variable
0,01,03,sospecha diagnostica,tos ferina
1,01,04,sospecha diagnostica,difteria
2,02,09,sospecha diagnostica,fiebre hemorrag. bol
3,02,14,sospecha diagnostica,enfer. por hanta virus
4,06,02,otras infecciones,ira sin neumonia
5,06,03,otras infecciones,neumonia
6,06,04,otras infecciones,malaria
7,06,05,otras infecciones,leishmaniasis
8,07,01,tuberculosis y lepra,sintomatico respiratorio
9,07,03,tuberculosis y lepra,tuberculosis pulmonar baar positivo nuevos


## Columnas con descripciones más significativas

Un problema con el proceso hasta este momento es cómo las columnas en cada set de datos tienen nombres difíciles de interpretar, resultado de aplanar las tablas que produce el sistema del snis en una forma uniforme. Para tener columnas más fáciles de interpretar infiero los atributos que representan y construyo un mejor nombre. Cada columna puede describir casos que ocurren dentro o fuera de establecimientos de salud, hacer referencia a hombres o mujeres, representar distintos rangos de edad y referirse a poblaciones o eventos particulares. Identifico estos atributos y construyo mejores nombres de columna que aplico a todos los documentos producidos.

In [61]:
patterns = { 
    
    'dentro':{
        'dentro de establecimientos': '(dentro)|(\-d[\-\_])|(Dentro)|(DENTRO)',
        'fuera de establecimientos': '(fuera)|(\-f[\-\_])|(Fuera)|(FUERA)'
    },
    
    'sexo': {
        'hombres': '(-H$)|(_masculino$)|(-m$)|(_masculino-1$)|(H1$)',
        'mujeres': '(-M$)|(_femenino$)|(-f$)|(_femenino-1$)|(M1$)|(mujer)'
    },
    
    'edad': {
        'menores a 1 año': '(< de 1)',
        'de 1 a 4 años': '(1 a 4)|(1-a-4-anos)|(1-4-anos)',
        'de 5 a 14 años': '(5 a 14)',
        'de 15 a 59 años': '(15 a 59)',
        'de 60 y más años': '(60 y más)|(60-y-mas)|(60-anos-y-mas)',
        'de 5 a 9 años': '(5 a 9)|(5-a-9-anos)|(5-9-anos)',
        'de 10 a 20 años': '(10 a 20)|(10-a-20-anos)',
        'de 21 a 59 años': '(21 a 59)|(21-a-59-anos)',
        'de 1 año': '(de-1-ano)',
        'menores a 2 años': '(< de 2)',
        'de 2 a 4 años': '(2 a 4)',
        'de 1 a 2 años': '(1-a-menor-de-2)',
        'de 2 a 5 años': '(2-a-menor-de-5)',
        'menores a 6 meses': '(menor-de-6-meses)',
        'de 6 meses a 1 año': '(6-meses-a-menor-de-1)|(6-m-a-menor-de-1-ano)',
        'de 10 a 14 años': '(10-14-anos)|(10-a-14-anos)',
        'de 15 a 19 años': '(15-19-anos)',
        'de 20 a 39 años': '(20-39-anos)',
        'de 40 a 49 años': '(40-49-anos)',
        'de 50 a 59 años': '(50-59-anos)',
        'de 6 meses': '(de-6-meses_)'
    },
    
    'especial': {
        'total': '(TOTAL)|(total)',
        'cantidad': '(Cantidad)',
        'embarazadas según IMC': '(imc_mujer-embarazada)',
        'población general según IMC': '(imc-1_)',
        'número de eventos': '(^Nro.$)|(Nro. Eventos$)',
        'número de afectados': '(^Nro. Afectados)|(Nro. Personas afectadas$)',
        'número de fallecidos': '(^Nro. Fallecidos)|(Nro. Personas fallecidas$)',
        
    }
}
    

def infer_category(text, patterns, lower=False):
    
    if lower:
        text = text.lower()
    
    category = None
    
    for k in patterns.keys():
        if len(re.findall(patterns[k], text)) > 0:
            category = k
    
    return category

def field_format(text):
    if text == None:
        return ''
    else:
        return text

In [62]:
## Contruyo una lista con todos los nombres de columnas en todos los sets de datos

indice = pd.read_csv('resources/indice.csv', dtype={'grvar': str, 'subvar': str})#no olvidarse espeificar el type de datos que son grvar y subvar
unicos = indice[['grvar', 'subvar', 'grupo', 'variable']].drop_duplicates(subset=['grvar', 'subvar', 'variable']).reset_index(drop=True)
unicos = unicos[unicos.variable.notna()]

schemas = []

for i, condicion in unicos.iterrows():
    
    for e, row in indice[(indice.grvar == condicion.grvar) & (indice.subvar == condicion.subvar) & (indice.variable == condicion.variable)].iterrows():
        schemas.append({**row, **{'columns': pd.read_csv('' + row.filename).columns.tolist()}})
        
schemas = pd.DataFrame(schemas)

columnas = pd.Series(list(itertools.chain.from_iterable(schemas['columns'].drop_duplicates().tolist()))).drop_duplicates().tolist()

## Infiero atributos en estos nombres: si se refiere a hombres o mujeres, si describe casos que ocurren dentro o fuera de establecimientos de salud, qué rangos de edad representan y si indican alguna población en particular

column_classes = pd.DataFrame({c: {k: infer_category(c, patterns[k]) for k in patterns.keys()} for c in columnas[6:]}).T##son 6 COLUMNAS

## Compongo una descripción con estos atributos

column_descriptions = column_classes.apply(lambda row: ' '.join(' '.join([field_format(field) for field in row]).split()), axis=1).to_dict()

In [63]:
## Guardo una tabla con los atributos y un diccionario con descripciones

column_classes.to_csv('resources/column_atributes.csv')

with open('resources/column_descriptions.json', 'w+') as f:
    json.dump(column_descriptions, f)

In [64]:
column_classes

,dentro,sexo,edad,especial
< de 1 año,None,None,menores a 1 año,None
1 a 4 años,None,None,de 1 a 4 años,None
5 a 14 años,None,None,de 5 a 14 años,None
15 a 59 años,None,None,de 15 a 59 años,None
60 y más,None,None,de 60 y más años,None
...,...,...,...,...
j-60-anos-y-mas-d-1_femenino,dentro de establecimientos,mujeres,de 60 y más años,None
j-60-anos-y-mas-f_masculino,fuera de establecimientos,hombres,de 60 y más años,None
j-60-anos-y-mas-f-1_femenino,fuera de establecimientos,mujeres,de 60 y más años,None
f-15-19-anos-f-2_masculino-1,fuera de establecimientos,hombres,de 15 a 19 años,None


In [65]:
## Remplazo los nombres de columnas en cada documento producido

for fn in indice.filename.tolist():
    filename = '' + fn
    dfi = pd.read_csv(filename)
    dfi.columns = [column_descriptions[c] if c in column_descriptions.keys() else c for c in dfi.columns]
    dfi.to_csv(filename, index=False)



In [67]:
pd.read_csv('{}'.format(indice.sample()['filename'].iloc[0]))


,DEPARTAMENTO,codigo_municipio,municipio,grupo,variable,year,menores a 1 año,de 1 a 4 años,de 5 a 14 años,de 15 a 59 años,de 60 y más años,total
0,POTOSI,51302,ACACIO,sospecha diagnostica,difteria,2006,0,0,0,0,0,0
1,LA PAZ,20201,ACHACACHI,sospecha diagnostica,difteria,2006,0,0,0,0,0,0
2,LA PAZ,20104,ACHOCALLA,sospecha diagnostica,difteria,2006,0,0,0,0,0,0
3,COCHABAMBA,30201,AIQUILE,sospecha diagnostica,difteria,2006,0,0,0,0,0,0
4,COCHABAMBA,31303,ALALAY,sospecha diagnostica,difteria,2006,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
320,SANTA CRUZ,70403,YAPACANI,sospecha diagnostica,difteria,2006,0,0,0,0,0,0
321,POTOSI,50103,YOCALLA,sospecha diagnostica,difteria,2006,0,0,0,0,0,0
322,CHUQUISACA,10102,YOTALA,sospecha diagnostica,difteria,2006,0,0,0,0,0,0
323,TARIJA,60402,YUNCHARA,sospecha diagnostica,difteria,2006,0,0,0,0,0,0


Esta tabla muestra los datos para casos clasificados en una variable y año, con enlaces a documentos csv:

In [68]:
def draw_table(df): 
    
    def link(dfi, name, link):
        return dfi[[link, name]].apply(lambda row: '<a href="{}">{}</a>'.format(row[0], row[1]), axis=1)
    
    dfi = df.copy()
    url_base = 'C:/Users/nerina/Documents/data_disease/'
    dfi['filename'] = dfi.filename.apply(lambda f: '{}{}'.format(url_base, f))
    dfi['variable'] = link(dfi, name='variable', link='filename')
    dfi = dfi[['year', 'grupo', 'variable']]
    dfi.columns = ['Año', 'Grupo', 'Variable']
    
    show(dfi, 
         order = [],
         hover = True,
         compact=True,
         scrollY="900px", 
         lengthMenu=[50,100],
         scrollCollapse=True,
         search={"caseInsensitive": True},
         paging=True,
         language={
             'lengthMenu': 'Mostrar _MENU_ filas',
             'search': '&#x1F50E;&#xFE0E;', 
             'processing': 'creando tabla ...', 
             'info': '', 
             'infoEmpty': '', 
             'infoFiltered':'_TOTAL_ documentos',
             'paginate': {
                'first': "Primero",
                'previous': "Anterior",
                'next': "Siguiente",
                'last': "Último"
            },
         }, 
         maxBytes=0,
         columnDefs=[
             {"width": "5px", "targets": [0]},
             {"width": "20px", "targets": [1]},
             {"width": "100px", "targets": [2]}
         ]
        )

draw_table(indice)